In [2]:
using Turing, DataFrames, Distributions, Random, Statistics, MCMCChains, LinearAlgebra

# ===========================
# 1. SIMULAÇÃO DE DADOS (COM NOVOS VALORES DE REGIME)
# ===========================
println(repeat("=", 80))
println("[1/6] SIMULANDO DADOS EM JULIA")
println(repeat("=", 80))

Random.seed!(1872)
anos = 1872:2022
n_anos = length(anos)

# Simulação das variáveis (usando broadcasting '.')
pib_pc = 500 .* exp.(LinRange(0, 1.5, n_anos)) .+ rand(Normal(0, 50), n_anos)
urbanizacao = 0.8 ./ (1 .+ exp.(-0.05 .* (anos .- 1990))) .+ rand(Normal(0, 0.02), n_anos)
urbanizacao = clamp.(urbanizacao, 0.05, 0.85)
industrializacao = 0.6 ./ (1 .+ exp.(-0.06 .* (anos .- 1980))) .+ rand(Normal(0, 0.02), n_anos)
educacao = 0.7 ./ (1 .+ exp.(-0.04 .* (anos .- 1985))) .+ rand(Normal(0, 0.015), n_anos)

# --- MODIFICAÇÃO SOLICITADA ---
gini_por_regime = Dict(0 => 0.40, 1 => 0.43, 2 => 0.46)
volatilidade_regime = Dict(0 => 0.025, 1 => 0.015, 2 => 0.020)

# Definição de regimes (forma idiomática em Julia)
regimes_verdadeiros = [
    if ano < 1930 0
    elseif 1930 <= ano < 1960 1
    elseif 1960 <= ano < 2000 0
    else 2
    end
    for ano in anos
]

# Recalculando o Gini
tendencia = 0.6 .- 0.10 .* ((anos .- 1872) ./ 150)
ciclica = -0.05 .* sin.(pi .* (anos .- 1872) ./ 40)
estrutural = 0.03 .* cos.(pi .* (anos .- 1872) ./ 25)

# Usando 'get' para buscar no Dicionário
gini_regime = [gini_por_regime[r] for r in regimes_verdadeiros]
noise_regime = [rand(Normal(0, volatilidade_regime[r])) for r in regimes_verdadeiros]

# Ajuste dos pesos
gini_real = 0.1 .* tendencia .+ 0.7 .* gini_regime .+ 0.1 .* ciclica .+ 0.1 .* estrutural .+ noise_regime
gini_real = clamp.(gini_real, 0.35, 0.55)

# Criando o DataFrame
df = DataFrame(
    Ano = anos,
    PIB_pc = pib_pc,
    Urbanizacao = urbanizacao,
    Industrializacao = industrializacao,
    Educacao = educacao,
    Gini_Verdadeiro = gini_real,
    Regime_Verdadeiro = regimes_verdadeiros
)

# Separando dados históricos e modernos
ano_inicio_pnAD = 1976
df_moderno = copy(df[df.Ano .>= ano_inicio_pnAD, :])
df_moderno.Gini_Observado = df_moderno.Gini_Verdadeiro .+ rand(Normal(0, 0.01), nrow(df_moderno))
df_historico = copy(df[df.Ano .< ano_inicio_pnAD, :])

n_historico = nrow(df_historico)
n_moderno = nrow(df_moderno)
n_total = nrow(df)

println("DADOS RE-SIMULADOS COM NOVOS REGIMES:")
println("Gini por Regime: $gini_por_regime")
println("Período histórico: $(minimum(df_historico.Ano))-$(maximum(df_historico.Ano)) ($n_historico anos)")
println("Período moderno: $(minimum(df_moderno.Ano))-$(maximum(df_moderno.Ano)) ($n_moderno anos)")


# ===========================
# 6. MODELO BAYESIANO HIERÁRQUICO (TURING.JL)
# ===========================
println("\n[6/6] MODELO BAYESIANO HIERÁRQUICO (TURING.JL)")

# Função de normalização
function normalizar(s)
    return (s .- mean(s)) ./ std(s)
end

df.PIB_norm = normalizar(df.PIB_pc)
df.Urb_norm = normalizar(df.Urbanizacao)
df.Ind_norm = normalizar(df.Industrializacao)
df.Edu_norm = normalizar(df.Educacao)
df.Ano_norm = normalizar(df.Ano)

# Preparando os dados para o Turing
preditores = ["PIB_norm", "Urb_norm", "Ind_norm", "Edu_norm"]
n_preditores = length(preditores)
X_data_total = Matrix(df[:, preditores]) # Converte para Matriz

# Índices (Turing prefere índices)
historico_idx = findall(df.Ano .< ano_inicio_pnAD)
moderno_idx = findall(df.Ano .>= ano_inicio_pnAD)
y_observado = df_moderno.Gini_Observado

# ---------------------------------------------
# Definição do Modelo Turing
# ---------------------------------------------
@model function modelo_bayes_avancado(X_data, y_data, moderno_idx, historico_idx, n_total, n_preditores)
    # Priors
    alpha ~ Normal(0.6, 0.1)
    
    # Prior para betas (MvNormal é mais eficiente que um loop)
    # ATENÇÃO: O nome do parâmetro é 'betas' (a palavra)
    betas ~ MvNormal(n_preditores, 0.5) 
    
    # Priors para sigmas (HalfNormal)
    sigma_temporal ~ Truncated(Normal(0, 0.02), 0, Inf)
    sigma ~ Truncated(Normal(0, 0.02), 0, Inf)

    # Tendência (Random Walk)
    tendencia_passos ~ MvNormal(n_total, sigma_temporal)
    tendencia = cumsum(tendencia_passos)
    
    # Cálculo de mu (determinístico)
    # X_data * betas faz a multiplicação matricial
    mu_total = alpha .+ X_data * betas .+ tendencia
    
    # Likelihood (Observações do período moderno)
    # y_data é o y_observado (df_moderno.Gini_Observado)
    y_data ~ MvNormal(mu_total[moderno_idx], sigma)
    
    # =================================================================
    # <<< SOLUÇÃO PARA O PROBLEMA 6 >>>
    #
    # Usamos `:=` para salvar variáveis determinísticas na 'chain'.
    # Isso é o equivalente ao `pm.Deterministic` do PyMC.
    # 'mu_hist' será calculado durante a amostragem e salvo
    # diretamente no objeto 'chain', evitando `generated_quantities`.
    # =================================================================
    mu_hist := mu_total[historico_idx]
end

# ---------------------------------------------
# Amostragem MCMC
# ---------------------------------------------

# Inicializando variáveis de fallback
bayesian_run_success = false
chain_avancado = nothing

# Pré-aloca as colunas no DataFrame histórico com o tipo correto
df_historico.Gini_Bayes_Hierarquico = Vector{Union{Missing, Float64}}(missing, n_historico)
df_historico.Gini_Bayes_Low = Vector{Union{Missing, Float64}}(missing, n_historico)
df_historico.Gini_Bayes_High = Vector{Union{Missing, Float64}}(missing, n_historico)

try
    # Instanciando o modelo com os dados
    model = modelo_bayes_avancado(
        X_data_total, 
        y_observado, 
        moderno_idx, 
        historico_idx, # Passando os índices históricos
        n_total, 
        n_preditores
    )

    println("→ Amostragem MCMC: 4 chains × 1000 warmup × 1000 amostras")
    println("   (target_accept = 0.99, Sampler = NUTS)")
    
    # Definindo o sampler: 1000 passos de warmup (adaptação), target_accept 0.99
    sampler = NUTS(1000, 0.99) 
    
    # Rodando a amostragem:
    # MCMCThreads() = usar threads paralelas (rode Julia com -t auto)
    # 1000 = número de amostras (pós-warmup)
    # 4 = número de cadeias
    global chain_avancado = sample(model, sampler, MCMCThreads(), 1000, 4, progress=true)
    
    println("✓ Amostragem concluída.")
    global bayesian_run_success = true

catch e
    # Em Julia, a concatenação de strings é feita com '*'
    println("\n" * repeat("=", 80)) ### <-- CORREÇÃO AQUI ###
    println("⚠ ERRO DURANTE A AMOSTRAGEM MCMC")
    println(e)
    println(repeat("=", 80))
    global bayesian_run_success = false
end

# ---------------------------------------------
# Sumário e Extração (Pós-amostragem)
# ---------------------------------------------

if bayesian_run_success && chain_avancado !== nothing
    # Em Julia, a concatenação de strings é feita com '*'
    println("\n" * repeat("=", 80)) ### <-- CORREÇÃO AQUI ###
    println("SUMÁRIO DOS PARÂMETROS DO MODELO BAYESIANO (POSTERIOR)")
    println(repeat("=", 80))
    
    # Mostra o sumário dos hiperparâmetros
    try
        # =================================================================
        # <<< CORREÇÃO PARA 'ArgumentError: index β not found' >>>
        #
        # O erro ocorre se 'vars' contiver :β (letra grega) em vez 
        # de :betas (o nome que demos ao parâmetro no @model).
        # A linha abaixo usa :betas, que é o correto.
        # =================================================================
        summary_stats = summarystats(chain_avancado, vars=[:alpha, :betas, :sigma_temporal, :sigma])
        println(summary_stats)
        
    catch e_summary
        println("Não foi possível gerar o sumário: $e_summary")
    end
        println(repeat("=", 80))

    # Bloco de extração robusto
    try
        println("→ Extraindo posteriores de 'mu_hist' diretamente da chain...")
        
        # Acessamos 'mu_hist' como qualquer outro parâmetro.
        y_bayes_post = chain_avancado[:mu_hist]

        # Calculando a média (MCMCChains 'mean' retorna um NamedTuple)
        # .nt.mean extrai o vetor de médias
        y_bayes_mean = vec(mean(y_bayes_post).nt.mean)
        
        # Calculando o HDI (High Density Interval)
        # 'hdi' retorna um DataFrame com colunas :lower e :upper
        hdi_bayes_df = hdi(y_bayes_post, prob=0.95)

        # Populando o DataFrame histórico
        df_historico.Gini_Bayes_Hierarquico = y_bayes_mean
        df_historico.Gini_Bayes_Low = hdi_bayes_df.lower
        df_historico.Gini_Bayes_High = hdi_bayes_df.upper
        
        println("✓ Modelo Bayesiano completo e resultados extraídos.")
    
    catch e_extract
        println("⚠ Erro ao extrair posteriores Bayesianos: $e_extract")
        println("→ Usando fallback (missing) para os resultados.")
    end

else
    # Se a amostragem MCMC falhou completamente
    println("⚠ Modelo Bayesiano falhou em rodar.")
    println("→ Usando fallback (missing) para os resultados.")
end

println("\nResultado da extração (primeiras 5 linhas de df_historico):")
println(first(df_historico[:, [:Ano, :Gini_Verdadeiro, :Gini_Bayes_Hierarquico, :Gini_Bayes_Low, :Gini_Bayes_High]], 5))

println("\nResultado da extração (últimas 5 linhas de df_historico):")
println(last(df_historico[:, [:Ano, :Gini_Verdadeiro, :Gini_Bayes_Hierarquico, :Gini_Bayes_Low, :Gini_Bayes_High]], 5))

[1/6] SIMULANDO DADOS EM JULIA
DADOS RE-SIMULADOS COM NOVOS REGIMES:
Gini por Regime: Dict(0 => 0.4, 2 => 0.46, 1 => 0.43)
Período histórico: 1872-1975 (104 anos)
Período moderno: 1976-2022 (47 anos)

[6/6] MODELO BAYESIANO HIERÁRQUICO (TURING.JL)
→ Amostragem MCMC: 4 chains × 1000 warmup × 1000 amostras
   (target_accept = 0.99, Sampler = NUTS)


Sampling (4 threads)   0%|█                             |  ETA: N/A
┌ Info: Found initial step size
│   ϵ = 0.00625
└ @ Turing.Inference C:\Users\daves\.julia\packages\Turing\nZgyL\src\mcmc\hmc.jl:216
┌ Info: Found initial step size
│   ϵ = 0.00625
└ @ Turing.Inference C:\Users\daves\.julia\packages\Turing\nZgyL\src\mcmc\hmc.jl:216
┌ Info: Found initial step size
│   ϵ = 0.0015625
└ @ Turing.Inference C:\Users\daves\.julia\packages\Turing\nZgyL\src\mcmc\hmc.jl:216
┌ Info: Found initial step size
│   ϵ = 0.00625
└ @ Turing.Inference C:\Users\daves\.julia\packages\Turing\nZgyL\src\mcmc\hmc.jl:216
Sampling (4 threads)   0%|█                             |  ETA: 0:39:34
Sampling (4 threads)   1%|█                             |  ETA: 0:25:45
Sampling (4 threads)   2%|█                             |  ETA: 0:22:22
Sampling (4 threads)   2%|█                             |  ETA: 0:22:19
Sampling (4 threads)   2%|█                             |  ETA: 0:19:27
Sampling (4 threads)   3%|█           

✓ Amostragem concluída.

SUMÁRIO DOS PARÂMETROS DO MODELO BAYESIANO (POSTERIOR)


Sampling (4 threads) 100%|██████████████████████████████| Time: 0:15:46


Summary Statistics (262 x 8)
→ Extraindo posteriores de 'mu_hist' diretamente da chain...
⚠ Erro ao extrair posteriores Bayesianos: ArgumentError("index mu_hist not found")
→ Usando fallback (missing) para os resultados.

Resultado da extração (primeiras 5 linhas de df_historico):
5×5 DataFrame
 Row │ Ano    Gini_Verdadeiro  Gini_Bayes_Hierarquico  Gini_Bayes_Low  Gini_Bayes_High 
     │ Int64  Float64          Float64?                Float64?        Float64?        
─────┼─────────────────────────────────────────────────────────────────────────────────
   1 │  1872         0.438541                 missing         missing          missing 
   2 │  1873         0.360297                 missing         missing          missing 
   3 │  1874         0.35                     missing         missing          missing 
   4 │  1875         0.35                     missing         missing          missing 
   5 │  1876         0.383194                 missing         missing          missing 
